In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from selenium.webdriver.common.by import By 
from selenium import webdriver

In [2]:

url = "https://uww.org/events"
driver = webdriver.Chrome()
driver.get(url)
driver.find_element(By.XPATH, "//select[@data-drupal-selector='edit-field-date-range-value']/option[text()='2021']").click()
driver.find_element(By.CSS_SELECTOR, "#edit-submit-events").click()

In [3]:
site_page = driver.page_source
soup = BeautifulSoup(site_page, 'html.parser')

In [ ]:
event_headlines = soup.find_all('tr')
world_cup_event_links = []
event_links = []
for event_headline in event_headlines:
    if event_headline.find('td', class_='views-field views-field-field-streaming-available').find('a'):
        event_page = event_headline.find('td', class_='views-field views-field-field-streaming-available').find('a')['href']
        event_links.append(event_page)
        # if 'World Cup' in event_headline.find('td', class_='views-field views-field-field-event-type').text:
        #     world_cup_event_links.append(event_page)

dates = []
places = []
levels = []
tournament_names = []
all_wrestling_types_clicked = []
all_wrestling_types_active = []
win_bys = []
first_opponents = []
second_opponents = []
first_opponents_nation = []
second_opponents_nation = []
first_opponents_score = []
second_opponents_score = []
weight_categories = []
stages = []
winners = []
video_links = []
page_links = []

for event_link in event_links:
    event_url = requests.get(event_link)
    event_page = BeautifulSoup(event_url.text, 'html.parser')
    if not event_page.find('div', class_="media media--type-image media--view-mode-big"):
       
        event_date_place = event_page.find('p', class_='title--20 c-margin--0 text-primary').find('strong').text
        event_date = event_date_place[:event_date_place.rindex('2')+1]    
        event_place = event_date_place[event_date_place.rindex('2')+1:].strip('\n ')
        wrestling_level = event_page.find('span', class_='field field--name-field-age field--type-entity-reference field--label-hidden field--items').text
        tournament_name = event_page.find('h1', 'title--50 c-margin--20').text
        
        wrestling_type = event_page.find('ul', class_="nav nav-tabs nav-tabs--med title--16")
        wrestling_types = [style.find('a')['href'] for style in wrestling_type.find_all('li')]
        wrestling_type_controller = 0
        for url in wrestling_types:
            wrestling_type_name_active = event_page.find('ul', class_="nav nav-tabs nav-tabs--med title--16").find('li', class_='active').text # CHECK THIS
            wrestling_type_name_clicked = event_page.find('ul', class_="nav nav-tabs nav-tabs--med title--16").find_all('li')[wrestling_type_controller].text # CHECK THIS
            wrestling_page_url = requests.get(url)
            event_page = BeautifulSoup(wrestling_page_url.text, 'html.parser')
            
            
            weights = event_page.find('ul', class_='nav nav-inline pull-right text-bold')
            weight_cats = [weight_cat.find('a')['href'] for weight_cat in weights.find_all('li')]
            wrestling_type_controller += 1
            for url in weight_cats:
                weight_cat_url = requests.get(url)
                weight_cat_page = BeautifulSoup(weight_cat_url.text, 'html.parser')
                current_weight_cat_name = weight_cat_page.find('ul', class_='nav nav-inline pull-right text-bold').find('li', class_='active').find('a').text  # CHECK THIS
                
                
                for stage_and_opps in weight_cat_page.find_all('div', class_='c-accordian__item active is-active'):
                    
                    stage = stage_and_opps.find('strong').text
                    opponents_details = stage_and_opps.find('div', class_='c-accordian__slave c-accordian__slave--small').find_all('tr')
                    
                    for opponent_detail in opponents_details:
                        if opponent_detail.find('table', class_='table table-simple'):
                            if opponent_detail.find('td', class_='text-center').find('a'):
                                video_link = url + opponent_detail.find('td', class_='text-center').find('a')['href']
                            else:
                                video_link = 'No video link of this opponents'
                            
                            opponents = opponent_detail.find('h3', class_='title--14').find_all('a') 
                            if len(opponents) == 2:
                                opponent_1 = opponents[0].text[:-6]
                                opponent_1_nation = opponents[0].text[-4:-1]
                                opponent_2 = opponents[1].text[:-6]
                                opponent_2_nation = opponents[1].text[-4:-1]
                                win_and_score = opponent_detail.find('td', class_='text-right').find('strong').text
                                score = win_and_score.split(',')[1]
                                opponent_1_score = score.split(' - ')[0]
                                opponent_2_score = score.split(' - ')[1]
                                win_by = win_and_score.split(',')[0].strip('\n ')[3:]
                                
                                winners.append(opponent_1)
                                dates.append(event_date)
                                places.append(event_place)
                                levels.append(wrestling_level)
                                tournament_names.append(tournament_name)
                                all_wrestling_types_clicked.append(wrestling_type_name_clicked)
                                all_wrestling_types_active.append(wrestling_type_name_active)
                                weight_categories.append(current_weight_cat_name)
                                stages.append(stage)
                                first_opponents.append(opponent_1)
                                second_opponents.append(opponent_2)
                                first_opponents_nation.append(opponent_1_nation)
                                second_opponents_nation.append(opponent_2_nation)
                                first_opponents_score.append(opponent_1_score)
                                second_opponents_score.append(opponent_2_score)
                                win_bys.append(win_by)
                                page_links.append(url)
                                video_links.append(video_link)
                            else:
                                continue
                        else:
                            continue
                     
wrestling_events_2021 = pd.DataFrame({'Tournament_Name': tournament_names,
                                 'Place': places,
                                 'Date': dates,
                                 'Level': levels,
                                 'CLICKED Wrestlig type': all_wrestling_types_clicked,
                                 'ACTIVE Wrestlig type': all_wrestling_types_active,
                                 'Weight': weight_categories,
                                 'Opponent 1': first_opponents,
                                 'Opponent 1 nation': first_opponents_nation,
                                 'Opponent 2': second_opponents,
                                 'Opponent 2 nation': second_opponents_nation,
                                 'Opponent 1 points': first_opponents_score,
                                 'Opponent 2 points': second_opponents_score,
                                 'Stage': stages,
                                 'Winner': winners,
                                 'Win by': win_bys,
                                 'Video links': video_links,
                                 'Page links': page_links})


In [8]:
wrestling_events_2021.to_excel('wrestling_events_2021.xlsx', index = False)